In [1]:
import pandas as pd
import requests as rq
import json
from datetime import datetime as dt
import ast

In [2]:
api = rq.get("https://api.cartolafc.globo.com/atletas/mercado")

In [3]:
market = json.loads(api.text)

In [4]:
market.keys()

dict_keys(['atletas', 'clubes', 'posicoes', 'status'])

In [21]:
round_num = '17'

In [22]:
roundN = pd.DataFrame(market["atletas"])

In [23]:
with open("../../status_id.json") as file:
    status_id = json.load(file)
with open("../../positions.json") as file:
    pos_id = json.load(file)

In [17]:
roundN = roundN[["rodada_id", "atleta_id", "nome", "apelido", "posicao_id", "clube_id", "scout", "pontos_num", "preco_num", "variacao_num"]]
roundN.columns = ["round_num", "player_id", "player_name", "player_nickname", "position", "team_id", "scout", "total_score", "current_price", "price_delta"]

In [18]:
roundN.position = roundN.position.apply(lambda x: pos_id[str(x)])

In [19]:
def scout_parser(scout, col_names):
    scout_dict = ast.literal_eval(scout)
    scout_list = []
    for col in col_names:
        try:
            el = scout_dict[col]
        except:
            el = 0
        scout_list.append(el)
    return pd.DataFrame(columns = col_names, data = [scout_list]).iloc[0]

In [11]:
col_names = ["G", "A", "FT", "FD", "FF", "FS", "PP", "I", "PE", "SG", "DP", "DD", "RB", "GC", "CV", "CA", "GS", "FC"]
roundN[col_names] = roundN.scout.apply(lambda x: scout_parser(str(x), col_names))
roundN = roundN[["round_num", "player_id", "player_name", "player_nickname", "position", "team_id", "G", "A", "FT", "FD", "FF", "FS", "PP", "I", "PE", "SG", "DP", "DD", "RB", "GC", "CV", "CA", "GS", "FC", "total_score", "current_price", "price_delta"]]

In [12]:
roundN.head(10)

,round_num,player_id,player_name,player_nickname,position,team_id,G,A,FT,FD,...,DD,RB,GC,CV,CA,GS,FC,total_score,current_price,price_delta
0,16,37673,Alecsandro Barbosa Felisbino,Alecsandro,ATA,341,0,0,0,4,...,0,1,0,0,0,0,8,0.70,4.25,-0.11
1,16,36940,Oswaldo de Oliveira,Oswaldo de Oliveira,TEC,266,0,0,0,0,...,0,0,0,0,0,0,0,0.00,6.90,0.00
2,16,37694,Henrique Pacheco de Lima,Henrique,MEI,283,0,0,0,2,...,0,30,0,0,0,0,16,3.90,8.26,0.24
3,16,37246,Ney Franco da Silveira Júnior,Ney Franco,TEC,290,0,0,0,0,...,0,0,0,0,0,0,0,3.73,8.86,0.11
4,16,37614,Evandro Goebel,Evandro,MEI,277,0,0,0,0,...,0,3,0,0,0,0,3,0.60,4.48,-0.03
5,16,37662,Jonathan Cícero Moreira,Jonathan,LAT,293,1,0,0,1,...,0,6,0,0,2,0,4,0.00,7.30,0.00
6,16,37656,Fábio Deivson Lopes Maciel,Fábio,GOL,283,0,0,0,0,...,16,0,0,0,1,17,0,-1.60,9.27,-1.68
7,16,50459,Leonardo Renan Simões de Lacerda,Léo,ZAG,283,0,0,0,0,...,0,15,0,0,1,0,14,0.00,7.01,0.00
8,16,37655,Rafael Martiniano de Miranda Moura,Rafael Moura,ATA,290,0,0,0,1,...,0,1,0,0,0,0,5,-0.30,3.39,-0.21
9,16,37657,Eduardo Luís Abonizio de Souza,Edu Dracena,ZAG,275,0,0,0,1,...,0,4,0,0,0,0,3,0.00,6.97,0.00


In [13]:
roundN.to_csv("2019_round"+ round_num + "_scouts.csv", index=False)

In [24]:
status = roundN[["rodada_id", "atleta_id", "status_id"]]

In [25]:
status.rodada_id = status.rodada_id + 1
now = dt.now()
status["query_time"] = dt(now.year, now.month, now.day, now.hour, now.minute, now.second)
status.status_id = status.status_id.apply(lambda x: status_id[str(x)])

C:\Users\fellipefcm\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\fellipefcm\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
status = status[["rodada_id", "query_time", "atleta_id", "status_id"]]
status.columns = ["round_num", "query_time", "player_id", "status_id"]
status.head()

,round_num,query_time,player_id,status_id
0,17,2019-08-29 10:28:59,37673,ProvÃ¡vel
1,17,2019-08-29 10:28:59,36940,ProvÃ¡vel
2,17,2019-08-29 10:28:59,37694,Nulo
3,17,2019-08-29 10:28:59,37246,ProvÃ¡vel
4,17,2019-08-29 10:28:59,37614,DÃºvida


In [27]:
filename = "../status/2019_round"
if status.round_num[0] < 10:
    filename = filename + "0" + str(status.round_num[0]) + "_status.csv"
else:
    filename = filename + str(status.round_num[0]) + "_status.csv"

status.to_csv(filename, index=False)